In [1]:
import torch

from utils import DEVICE, init_random_seeds
from models._utils import (
    print_parameters,
    LinearRegression,
    FeatureFFNN,
    NeuralTransformer,
    NetworkLSTM,
)
from fvcore.nn import (
    FlopCountAnalysis,
    ActivationCountAnalysis,
    flop_count_table,
    flop_count_str,
)

# Display the DEVICE
print(f"DEVICE: {DEVICE}")

# Initialize the random seeds
init_random_seeds(0)

CUDA device found.
DEVICE: cuda


In [2]:
# @title Prepare model and input
# @markdown Make sure the model and input are on the same device.

# Set shapes for model and input
seq_len = 100
input_size = 208
hidden_size = 512

# Use a standard PyTorch model
model = torch.nn.Linear(in_features=input_size, out_features=hidden_size)
model = model.to(DEVICE)
model.eval()  # switch to eval mode
print(f"Simple PyTorch model: {model}\n")

# Create input of the correct shape for the model
input = torch.randn(1, seq_len, input_size).to(DEVICE)  # batch_size=1
mask = None
print(f"Input: {input.shape} \t Output: {model(input).shape}", end="\n\n")
print(f"\n{'~'*100}\n")

# Load one of our custom models instead
model_args = dict(input_size=input_size, hidden_size=hidden_size, loss="MSE")
model = LinearRegression(**model_args)
# model = FeatureFFNN(**model_args)
# model = NeuralTransformer(**model_args)
# model = NetworkLSTM(**model_args)
model = model.to(DEVICE)
model.eval()  # switch to eval mode
print(f"Custom model: {model}\n")

# Create input of the correct shape for the model
input = torch.randn(1, seq_len, input_size).to(DEVICE)  # batch_size=1
mask = torch.ones(1, input_size).to(bool).to(DEVICE)
print(
    f"Input: {input.shape} \t Mask: {mask.shape} \t Output: {model(input, mask).shape}",
    end="\n\n",
)

Simple PyTorch model: Linear(in_features=208, out_features=512, bias=True)

Input: torch.Size([1, 100, 208]) 	 Output: torch.Size([1, 100, 512])


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Custom model: LinearRegression(
  (identity): Identity()
  (input_hidden): Identity()
  (hidden_hidden): Identity()
  (inner_hidden_model): InnerHiddenModel(
    (hidden_hidden): Identity()
  )
  (linear): Linear(in_features=208, out_features=208, bias=True)
  (layer_norm): LayerNorm((208,), eps=1e-05, elementwise_affine=True)
)

Input: torch.Size([1, 100, 208]) 	 Mask: torch.Size([1, 208]) 	 Output: torch.Size([1, 100, 208])



In [3]:
# @title Using fvcore

# Adjust input based on if we use standard PyTorch model or custom model
input = (input, mask) if mask is not None else input

# Count the total and number of trainable parameters
all_params_ct, train_params_ct = print_parameters(model)

print(f"\nAll params: {all_params_ct}\nTrainable params: {train_params_ct}", end="\n\n")

# Perform FLOP Counting: Use the FlopCountAnalysis class to analyze your model:
flops = FlopCountAnalysis(model, input)

# Print Results: You can now print out the FLOP and parameter information:
print(
    f"FLOP: {flops.total(), flops.by_operator(), flops.by_module(), flops.by_module_and_operator()}",
    end="\n\n",
)
print(flop_count_str(flops), end="\n\n")
print(flop_count_table(flops), end="\n\n")
print(
    f"\tParams: {sum(p.numel() for p in model.parameters() if p.requires_grad)}",
    end="\n\n",
)

# Perform Activations Counting: Use the ActivationCountAnalysis class to analyze your model:
acts = ActivationCountAnalysis(model, input)

# Print Results: You can now print out the FLOP and parameter information:
print(
    f"Activations: {acts.total(), acts.by_operator(), acts.by_module(), acts.by_module_and_operator()}",
    end="\n\n",
)
print(
    f"\tParams: {sum(p.numel() for p in model.parameters() if p.requires_grad)}",
    end="\n\n",
)

Unsupported operator aten::expand_as encountered 1 time(s)
Unsupported operator aten::mul encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
inner_hidden_model, layer_norm
Unsupported operator aten::expand_as encountered 1 time(s)
Unsupported operator aten::mul encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
inner_hidden_model, layer_norm



All params: 43888
Trainable params: 43888

FLOP: (4326400, Counter({'linear': 4326400}), Counter({'': 4326400, 'linear': 4326400, 'identity': 0, 'input_hidden': 0, 'hidden_hidden': 0, 'inner_hidden_model': 0, 'layer_norm': 0}), {'': Counter({'linear': 4326400}), 'identity': Counter(), 'input_hidden': Counter(), 'hidden_hidden': Counter(), 'inner_hidden_model': Counter(), 'linear': Counter({'linear': 4326400}), 'layer_norm': Counter()})

N/A indicates a possibly missing statistic due to how the module was called. Missing values are still included in the parent's total.
LinearRegression(
  #params: 43.89K, #flops: 4.33M
  (identity): Identity(#params: 0, #flops: N/A)
  (input_hidden): Identity(#params: 0, #flops: N/A)
  (hidden_hidden): Identity(#params: 0, #flops: N/A)
  (inner_hidden_model): InnerHiddenModel(
    #params: 0, #flops: N/A
    (hidden_hidden): Identity()
  )
  (linear): Linear(
    in_features=208, out_features=208, bias=True
    #params: 43.47K, #flops: 4.33M
  )
  (lay